<a href="https://colab.research.google.com/github/HegdeSiddesh/cs6910_Assignment3/blob/main/ASG_3_Best_model_without_attention.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
import pandas as pd
!pip install --upgrade wandb
import wandb
from wandb.keras import WandbCallback
from google.colab import files
import time
import math

In [ ]:
#Download and unzip the Dakshina dataset 
!wget https://storage.googleapis.com/gresearch/dakshina/dakshina_dataset_v1.0.tar
!tar -xf '/content/dakshina_dataset_v1.0.tar'

--2022-05-02 13:29:50--  https://storage.googleapis.com/gresearch/dakshina/dakshina_dataset_v1.0.tar
Resolving storage.googleapis.com (storage.googleapis.com)... 142.250.153.128, 142.250.145.128, 173.194.69.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|142.250.153.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2008340480 (1.9G) [application/x-tar]
Saving to: ‘dakshina_dataset_v1.0.tar’

dakshina_dataset_v1 100%[===================>]   1.87G   128MB/s    in 16s     

2022-05-02 13:30:06 (121 MB/s) - ‘dakshina_dataset_v1.0.tar’ saved [2008340480/2008340480]



In [ ]:
def load_data(path):
    with open(path) as fil:
        data = pd.read_csv(fil,sep='\t',header=None,names=["Tamil","English",""],skip_blank_lines=True,index_col=None)
    data = data[data['Tamil'].notna()]
    data = data[data['English'].notna()]
    data = data[['Tamil','English']]
    return data

train = load_data("/content/dakshina_dataset_v1.0/ta/lexicons/ta.translit.sampled.train.tsv")
val = load_data("/content/dakshina_dataset_v1.0/ta/lexicons/ta.translit.sampled.dev.tsv")
test = load_data("/content/dakshina_dataset_v1.0/ta/lexicons/ta.translit.sampled.test.tsv")


In [ ]:
x = train['English'].values
y = train['Tamil'].values
# We use "tab" as the "start sequence" character for the targets, and "\n" as "end sequence" character.
y = "\t"+y+"\n"

In [ ]:
input_characters = set()
target_characters = set()

for i in range(len(x)):
  input_characters.update(list(str(x[i])))
  target_characters.update(list(str(y[i])))

  # input_characters=list(set(input_characters+list(str(x[i]))))
  # target_characters=list(set(target_characters+list(str(y[i]))))

input_characters = sorted(list(input_characters))
target_characters = sorted(list(target_characters))

# add the space character to both
input_characters.append(" ")
target_characters.append(" ")

num_encoder_tokens = len(input_characters)
num_decoder_tokens = len(target_characters)

input_length=[]
target_length=[]
for i in range(len(x)):
  input_length.append(len(str(x[i])))
  target_length.append(len(str(y[i])))

max_encoder_seq_length = max(input_length)
max_decoder_seq_length = max(target_length)

print("Number of samples:", len(x))
print("Number of unique input tokens:", num_encoder_tokens)
print("Number of unique output tokens:", num_decoder_tokens)
print("Max sequence length for inputs:", max_encoder_seq_length)
print("Max sequence length for outputs:", max_decoder_seq_length)

input_token_index = dict([(char, i) for i, char in enumerate(input_characters)])
target_token_index = dict([(char, i) for i, char in enumerate(target_characters)])
print(input_token_index)
print(target_token_index)
encoder_input_data = np.zeros((len(x), max_encoder_seq_length, num_encoder_tokens), dtype="float32")
decoder_input_data = np.zeros((len(x), max_decoder_seq_length, num_decoder_tokens), dtype="float32")
decoder_target_data = np.zeros((len(x), max_decoder_seq_length, num_decoder_tokens), dtype="float32")

for i, (input_text, target_text) in enumerate(zip(x, y)):
    for t, char in enumerate(input_text):
        encoder_input_data[i, t, input_token_index[char]] = 1.0
    encoder_input_data[i, t + 1 :, input_token_index[" "]] = 1.0
    for t, char in enumerate(target_text):
        # decoder_target_data is ahead of decoder_input_data by one timestep
        decoder_input_data[i, t, target_token_index[char]] = 1.0
        if t > 0:
            # decoder_target_data will be ahead by one timestep
            # and will not include the start character.
            decoder_target_data[i, t - 1, target_token_index[char]] = 1.0
    decoder_input_data[i, t + 1 :, target_token_index[" "]] = 1.0
    decoder_target_data[i, t:, target_token_index[" "]] = 1.0

Number of samples: 68215
Number of unique input tokens: 27
Number of unique output tokens: 49
Max sequence length for inputs: 30
Max sequence length for outputs: 28
{'a': 0, 'b': 1, 'c': 2, 'd': 3, 'e': 4, 'f': 5, 'g': 6, 'h': 7, 'i': 8, 'j': 9, 'k': 10, 'l': 11, 'm': 12, 'n': 13, 'o': 14, 'p': 15, 'q': 16, 'r': 17, 's': 18, 't': 19, 'u': 20, 'v': 21, 'w': 22, 'x': 23, 'y': 24, 'z': 25, ' ': 26}
{'\t': 0, '\n': 1, 'ஃ': 2, 'அ': 3, 'ஆ': 4, 'இ': 5, 'ஈ': 6, 'உ': 7, 'ஊ': 8, 'எ': 9, 'ஏ': 10, 'ஐ': 11, 'ஒ': 12, 'ஓ': 13, 'க': 14, 'ங': 15, 'ச': 16, 'ஜ': 17, 'ஞ': 18, 'ட': 19, 'ண': 20, 'த': 21, 'ந': 22, 'ன': 23, 'ப': 24, 'ம': 25, 'ய': 26, 'ர': 27, 'ற': 28, 'ல': 29, 'ள': 30, 'ழ': 31, 'வ': 32, 'ஷ': 33, 'ஸ': 34, 'ஹ': 35, 'ா': 36, 'ி': 37, 'ீ': 38, 'ு': 39, 'ூ': 40, 'ெ': 41, 'ே': 42, 'ை': 43, 'ொ': 44, 'ோ': 45, 'ௌ': 46, '்': 47, ' ': 48}


In [ ]:
#Embedding test data
x_test = test['English'].values
y_test = test['Tamil'].values
# We use "tab" as the "start sequence" character for the targets, and "\n" as "end sequence" character.
y_test = "\t"+y_test+"\n"

In [ ]:
reverse_input_char_index = dict((i, char) for char, i in input_token_index.items())
reverse_target_char_index = dict((i, char) for char, i in target_token_index.items())

In [ ]:
test_input_characters = set()
test_target_characters = set()

for i in range(len(x_test)):
  test_input_characters.update(list(str(x_test[i])))
  test_target_characters.update(list(str(y_test[i])))

  # input_characters=list(set(input_characters+list(str(x[i]))))
  # target_characters=list(set(target_characters+list(str(y[i]))))

test_input_characters = sorted(list(test_input_characters))
test_target_characters = sorted(list(test_target_characters))

# add the space character to both
test_input_characters.append(" ")
test_target_characters.append(" ")

test_num_encoder_tokens = len(test_input_characters)
test_num_decoder_tokens = len(test_target_characters)

test_input_length=[]
test_target_length=[]
for i in range(len(x_test)):
  test_input_length.append(len(str(x_test[i])))
  test_target_length.append(len(str(y_test[i])))

test_max_encoder_seq_length = max(test_input_length)
test_max_decoder_seq_length = max(test_target_length)

print("Number of samples in test set:", len(x_test))
print("Number of unique input tokens:", test_num_encoder_tokens)
print("Number of unique output tokens:", test_num_decoder_tokens)
print("Max sequence length for test inputs:", test_max_encoder_seq_length)
print("Max sequence length for test outputs:", test_max_decoder_seq_length)

# val_input_token_index = dict([(char, i) for i, char in enumerate(val_input_characters)])
# val_target_token_index = dict([(char, i) for i, char in enumerate(val_target_characters)])
# print(val_input_token_index)
# print(val_target_token_index)

test_encoder_input_data = np.zeros((len(x_test), test_max_encoder_seq_length,test_num_encoder_tokens), dtype="float32")
test_decoder_input_data = np.zeros((len(x_test), test_max_decoder_seq_length,test_num_decoder_tokens), dtype="float32")
test_decoder_target_data = np.zeros((len(x_test), max_decoder_seq_length, test_num_decoder_tokens), dtype="float32")

for i, (input_text, target_text) in enumerate(zip(x_test, y_test)):
    for t, char in enumerate(input_text):
        test_encoder_input_data[i, t ,input_token_index[char]] = 1.0
    test_encoder_input_data[i, t + 1 :,input_token_index[" "]] = 1.0
    for t, char in enumerate(target_text):
        # decoder_target_data is ahead of decoder_input_data by one timestep
        test_decoder_input_data[i, t ,target_token_index[char]] = 1.0
        if t > 0:
            # decoder_target_data will be ahead by one timestep
            # and will not include the start character.
            test_decoder_target_data[i, t - 1, target_token_index[char]] = 1.0
    test_decoder_input_data[i, t + 1 :,target_token_index[" "]] = 1.0
    test_decoder_target_data[i, t:, target_token_index[" "]] = 1.0

Number of samples in test set: 6864
Number of unique input tokens: 27
Number of unique output tokens: 49
Max sequence length for test inputs: 23
Max sequence length for test outputs: 24


In [ ]:
# Using label encoding for the encoder inputs (and then find an embedding using the Embedding layer)
encoder_input_data = np.argmax(encoder_input_data, axis=2)

test_encoder_input_data = np.argmax(test_encoder_input_data, axis=2)
print(test_encoder_input_data.shape)

(6864, 23)


In [ ]:
decoder_input_data = np.argmax(decoder_input_data, axis=2)
test_decoder_input_data = np.argmax(test_decoder_input_data, axis=2)

In [ ]:
#Build the model
def create_model(num_encoder_tokens,embedding_size,cell_type,latent_dimension,dropout,number_of_encoder_layers,num_decoder_tokens,number_of_decoder_layers):

  # Define an input sequence and process it.
  encoder_inputs = keras.Input(shape=(None,), name='encoder_input')
  encoder = None
  encoder_outputs = None
  state_h = None
  state_c = None
  en_embed = tf.keras.layers.Embedding(input_dim=num_encoder_tokens, output_dim=embedding_size,
                                            name='encoder_embedding')(encoder_inputs)
  if cell_type == 'rnn':
      encoder = keras.layers.SimpleRNN(latent_dimension, return_state=True, return_sequences=True,
                                       name='encoder_hidden_1', dropout=dropout)
      encoder_outputs, state_h = encoder(en_embed)
  elif cell_type == 'gru':
      encoder = keras.layers.GRU(latent_dimension, return_state=True, return_sequences=True,
                                  name='encoder_hidden_1', dropout=dropout)
      encoder_outputs, state_h = encoder(en_embed)
  else:
      encoder = keras.layers.LSTM(latent_dimension, return_state=True, return_sequences=True,
                                  name='encoder_hidden_1', dropout=dropout)
      encoder_outputs, state_h, state_c = encoder(en_embed)

  # 1st layer
  # number of encoder layers
  e_layer = number_of_encoder_layers
  for i in range(2, e_layer + 1):
    # give the output sequences as input to the next layer also the last state is set as initial state of
    # next layer
    layer_name = ('encoder_hidden_%d') % i
    if cell_type == 'rnn':
      encoder = keras.layers.SimpleRNN(latent_dimension, return_state=True, return_sequences=True,
                                          name=layer_name, dropout=dropout)
      encoder_outputs, state_h = encoder(encoder_outputs, initial_state=[state_h])
    elif cell_type == 'gru':
        encoder = keras.layers.GRU(latent_dimension, return_state=True, return_sequences=True,
                                    name=layer_name, dropout=dropout)
        encoder_outputs, state_h = encoder(encoder_outputs, initial_state=[state_h])
    else:
        encoder = keras.layers.LSTM(latent_dimension, return_state=True, return_sequences=True,
                                    name=layer_name, dropout=dropout)
        encoder_outputs, state_h, state_c = encoder(encoder_outputs, initial_state=[state_h, state_c])
  
  encoder_states = None
  # save the last state
  if cell_type  == 'rnn' or cell_type == 'gru' :
      encoder_states = [state_h]
  else:
      encoder_states = [state_h, state_c]
  decoder_inputs = keras.Input(shape=(None,), name='decoder_input')
  de_embed = tf.keras.layers.Embedding(num_decoder_tokens, embedding_size, name='decoder_embedding')(decoder_inputs)
 # number of decoder layers
  d_layer = number_of_decoder_layers
  decoder = None
  # first layer
  if cell_type == 'rnn':
      decoder = keras.layers.SimpleRNN(latent_dimension, return_sequences=True, return_state=True,
                                        name='decoder_hidden_1', dropout=dropout)
      # all decoders the initial state is encoder last state of last layer
      decoder_outputs, _ = decoder(de_embed, initial_state=encoder_states)
  elif cell_type == 'gru':
      decoder = keras.layers.GRU(latent_dimension, return_sequences=True, return_state=True,
                                  name='decoder_hidden_1', dropout=dropout)
      # all decoders the initial state is encoder last state of last layer
      decoder_outputs, _ = decoder(de_embed, initial_state=encoder_states)
  else:
      decoder = keras.layers.LSTM(latent_dimension, return_sequences=True, return_state=True,
                                  name='decoder_hidden_1', dropout=dropout)
      # all decoders the initial state is encoder last state of last layer
      decoder_outputs, _, _ = decoder(de_embed, initial_state=encoder_states)

  for i in range(2, d_layer + 1):
      layer_name = 'decoder_hidden_%d' % i
      if cell_type == 'rnn':
          decoder = keras.layers.SimpleRNN(latent_dimension, return_sequences=True, return_state=True,
                                            name=layer_name, dropout=dropout)
          decoder_outputs, _ = decoder(decoder_outputs, initial_state=encoder_states)
      elif cell_type == 'gru':
          decoder = keras.layers.GRU(latent_dimension, return_sequences=True, return_state=True,
                                      name=layer_name, dropout=dropout)
          decoder_outputs, _ = decoder(decoder_outputs, initial_state=encoder_states)
      else:
          decoder = keras.layers.LSTM(latent_dimension, return_sequences=True, return_state=True,
                                      name=layer_name, dropout=dropout)
          decoder_outputs, _, _ = decoder(decoder_outputs, initial_state=encoder_states)

  # add a dense layer
  decoder_dense = keras.layers.Dense(num_decoder_tokens, activation="softmax", name='decoder_output')
  decoder_outputs = decoder_dense(decoder_outputs)

  model = keras.Model([encoder_inputs, decoder_inputs], decoder_outputs)
  
  return model
  # # We discard `encoder_outputs` and only keep the states.
  # encoder_states = [state_h, state_c]

  # # Set up the decoder, using `encoder_states` as initial state.
  # decoder_inputs = keras.Input(shape=(None, num_decoder_tokens))

  # # We set up our decoder to return full output sequences,
  # # and to return internal states as well. We don't use the
  # # return states in the training model, but we will use them in inference.
  # decoder_lstm = keras.layers.LSTM(latent_dim, return_sequences=True, return_state=True)
  # decoder_outputs, _, _ = decoder_lstm(decoder_inputs, initial_state=encoder_states)
  # decoder_dense = keras.layers.Dense(num_decoder_tokens, activation="softmax")
  # decoder_outputs = decoder_dense(decoder_outputs)

  # # Define the model that will turn
  # # `encoder_input_data` & `decoder_input_data` into `decoder_target_data`
  # model = keras.Model([encoder_inputs, decoder_inputs], decoder_outputs)

In [ ]:
def fit(model,cell_type,encoder_input_data, decoder_input_data, decoder_target_data,batch_size, epochs,number_of_encoder_layers,number_of_decoder_layers,latent_dimension, callbacks=None):
  model.compile(optimizer="adam", loss="categorical_crossentropy",metrics=['accuracy'])
  model.fit(
      [encoder_input_data, decoder_input_data],
      decoder_target_data,
      batch_size=batch_size,
      epochs=epochs,
      callbacks=callbacks
  )

  # create inference model
  encoder_inputs = model.input[0]  # input_1
  if cell_type == 'rnn' or cell_type == 'gru':
      encoder_outputs, state_h_enc = model.get_layer('encoder_hidden_' + str(number_of_encoder_layers)).output
      encoder_states = [state_h_enc]
      encoder_model = keras.Model(encoder_inputs, encoder_states)

      decoder_inputs = model.input[1]  # input_2
      decoder_outputs = model.get_layer('decoder_embedding')(decoder_inputs)
      decoder_states_inputs = []
      decoder_states = []

      for j in range(1, number_of_decoder_layers + 1):
          decoder_state_input_h = keras.Input(shape=(latent_dimension,))
          current_states_inputs = [decoder_state_input_h]
          decoder = model.get_layer('decoder_hidden_' + str(j))
          decoder_outputs, state_h_dec = decoder(decoder_outputs, initial_state=current_states_inputs)
          decoder_states += [state_h_dec]
          decoder_states_inputs += current_states_inputs
  else:
      encoder_outputs, state_h_enc, state_c_enc = model.get_layer('encoder_hidden_'+ str(number_of_encoder_layers)).output
      encoder_states = [state_h_enc, state_c_enc]
      encoder_model = keras.Model(encoder_inputs, encoder_states)

      decoder_inputs = model.input[1]  # input_2
      decoder_outputs = model.get_layer('decoder_embedding')(decoder_inputs)
      decoder_states_inputs = []
      decoder_states = []

      for j in range(1,number_of_decoder_layers + 1):
          decoder_state_input_h = keras.Input(shape=(latent_dimension,))
          decoder_state_input_c = keras.Input(shape=(latent_dimension,))
          current_states_inputs = [decoder_state_input_h, decoder_state_input_c]
          decoder = model.get_layer('decoder_hidden_' + str(j))
          decoder_outputs, state_h_dec, state_c_dec = decoder(decoder_outputs, initial_state=current_states_inputs)
          decoder_states += [state_h_dec, state_c_dec]
          decoder_states_inputs += current_states_inputs
          
  decoder_dense = model.get_layer('decoder_output')
  decoder_outputs = decoder_dense(decoder_outputs)
  decoder_model = keras.Model([decoder_inputs] + decoder_states_inputs, [decoder_outputs] + decoder_states)

  return encoder_model , decoder_model

In [ ]:
# Reverse-lookup token index to decode sequences back to
# something readable.
reverse_input_char_index = dict((i, char) for char, i in input_token_index.items())
reverse_target_char_index = dict((i, char) for char, i in target_token_index.items())

In [ ]:
def decode_sequence(input_seq,number_of_decoder_layers,target_token_index,cell_type,
                    reverse_target_char_index,max_decoder_seq_length,
                    encoder_model,decoder_model):
  # Encode the input as state vectors.
  states_value = [encoder_model.predict(input_seq)]*number_of_decoder_layers

  # Generate empty target sequence of length 1.
  empty_seq = np.zeros((1, 1))
  # Populate the first character of target sequence with the start character.
  empty_seq[0, 0] = target_token_index["\t"]
  target_seq = empty_seq

  # Sampling loop for a batch of sequences
  # (to simplify, here we assume a batch of size 1).
  stop_condition = False
  decoded_sentence = ""
  while not stop_condition:
      if cell_type  == 'rnn' or cell_type == 'gru':
          temp = decoder_model.predict([target_seq] + [states_value])
          output_tokens, states_value = temp[0], temp[1:]
      else:
          temp = decoder_model.predict([target_seq] + states_value )
          output_tokens, states_value = temp[0], temp[1:]

      # Sample a token
      sampled_token_index = np.argmax(output_tokens[0, -1, :])
      sampled_char = reverse_target_char_index[sampled_token_index]
      decoded_sentence += sampled_char

      # Exit condition: either hit max length
      # or find stop character.
      if sampled_char == "\n" or len(decoded_sentence) > max_decoder_seq_length:
          stop_condition = True

      # Update the target sequence (of length 1).
      target_seq = np.zeros((1, 1))
      target_seq[0, 0] = sampled_token_index

  return decoded_sentence

In [ ]:
def accuracy(test_encoder_input_data, y_test,number_of_decoder_layers,target_token_index,cell_type,
                    reverse_target_char_index,max_decoder_seq_length,
                    encoder_model,decoder_model,verbose=True,x_test):
  n_correct = 0
  n_total = 0
  number_of_decoder_layers=number_of_decoder_layers
  for seq_index in range(len(test_encoder_input_data)):

    # Take one sequence (part of the training set)
    # for trying out decoding.
    input_seq = test_encoder_input_data[seq_index: seq_index + 1]
    decoded_sentence = decode_sequence(input_seq,number_of_decoder_layers,target_token_index,cell_type,
                    reverse_target_char_index,max_decoder_seq_length,
                    encoder_model,decoder_model)
    # print("-")
    # print("Input sentence:", y_val[seq_index])
    # print("Decoded sentence:", decoded_sentence)

    if decoded_sentence.strip() == y_test[seq_index].strip():
        n_correct += 1

    n_total += 1
    row = {}
    row['SourceText'] = x_test[seq_index].strip()
    row['GroundTruth'] = y_test[seq_index].strip()
    row['Prediction'] = decoded_sentence[0][0].strip()
    df = df.append(row, ignore_index=True)
    df.to_csv('predictions_'+str(1)+'.csv', index=False)  
    
    if verbose:
        print('Prediction ', decoded_sentence.strip(), ',Ground Truth ', y_test[seq_index].strip())

  return n_correct * 100.0 / n_total

In [ ]:
def beam_search(input_seq, beam_size,target_token_index,encoder_model,number_of_decoder_layers,reverse_target_char_index,cell_type,
                deocder_model,max_decoder_seq_length):
  sequences = [([target_token_index["\t"]], 0.0)]
  # Encode the input as state vectors.
  states_value = [[encoder_model.predict(input_seq)]*number_of_decoder_layers]

  stop_condition = False
  t = 0
  while not stop_condition:
      all_seq = list()
      char_sequences = []
      for seq, score in sequences:
          char_seq = ''
          for index in seq:
              char_seq += reverse_target_char_index[index]
          char_sequences.append((char_seq, score))
      #print('at time ', t, char_sequences)
      t += 1
      for i in range(len(sequences)):
          seq, score = sequences[i]
          if seq[-1] == target_token_index["\n"] or seq[-1] == target_token_index[" "]:
              all_seq.append((seq, score))
              continue
          target_seq = np.zeros((1, 1))
          target_seq[0, 0] = seq[-1]
          # print('target seq', seq[-1], self.reverse_target_char_index[seq[-1]])
          if cell_type == 'rnn' or cell_type == 'gru':
              temp = decoder_model.predict([target_seq] + [states_value[i]])
              output_tokens, temp_states = temp[0], temp[1:]
          else:
              temp = decoder_model.predict([target_seq] + states_value[i] )
              output_tokens, temp_states = temp[0], temp[1:]

          if t == 1:
              states_value = [temp_states] * beam_size
          else:
              states_value[i] = temp_states

          for j in range(len(output_tokens[0, -1, :])):
              candidate = (seq + [j], score - math.log(output_tokens[0, -1, j]))
              all_seq.append(candidate)

          # Exit condition: either hit max length
          # or find stop character.
          sampled_token_index = np.argmax(output_tokens[0, -1, :])
          sampled_char = reverse_target_char_index[sampled_token_index]
          # print('prob', output_tokens[0, -1, :])
          # print('sampledchar ', sampled_char)

      sorted_by_prob = sorted(all_seq, key=lambda tup: tup[1])

      # print all possible sequences
      char_sequences = []
      for seq, score in sequences:
          char_seq = ''
          for index in seq:
              char_seq += reverse_target_char_index[index]
          char_sequences.append((char_seq, score))
      # print('Printing all sequences')
      # print(char_sequences)

      # select the top k sequences
      sequences = sorted_by_prob[:beam_size]
      if t > max_decoder_seq_length:
          stop_condition = True
      # if every sequence has predicted \n we should stop
      all_seq_ended = True
      for seq, _ in sequences:
          if seq[-1] != target_token_index["\n"]:
              all_seq_ended = False
              break
      if all_seq_ended:
          stop_condition = True
  # create character out of indexes
  char_sequences = []
  for seq, score in sequences:
      char_seq = ''
      for index in seq:
          char_seq += reverse_target_char_index[index]
      char_sequences.append((char_seq, score))
  return char_sequences

In [ ]:

batch_size = 64  # Batch size for training.
epochs = 20  # Number of epochs to train for.
latent_dim = 512  # Latent dimensionality of the encoding space .#hidden states hyperparameter

#Train the model
model = create_model(num_encoder_tokens=num_encoder_tokens,embedding_size=256,
                  cell_type = 'gru',latent_dimension=512,
                  dropout=0.50,number_of_encoder_layers = 1,
                  num_decoder_tokens=num_decoder_tokens,
                  number_of_decoder_layers=3)

encoder_model , decoder_model = fit(model=model,cell_type='gru',encoder_input_data=encoder_input_data, 
                                decoder_input_data=decoder_input_data,
      decoder_target_data=decoder_target_data,batch_size=batch_size,epochs=epochs,
      number_of_encoder_layers = 1,
      number_of_decoder_layers= 3,latent_dimension=latent_dim, 
      )

#Test accuracy

subset = 20

test_accuracy = accuracy(test_encoder_input_data[0:subset], y_test[0:subset],
                      number_of_decoder_layers=3,
                      target_token_index=target_token_index,cell_type='gru',
                      reverse_target_char_index=reverse_target_char_index,max_decoder_seq_length=max_decoder_seq_length,
                      encoder_model=encoder_model,decoder_model=decoder_model
                      )

print('Test accuracy: ', test_accuracy)


# print('Calculating test accuracy')
# test_accuracy = {}
# for beamSize in range(1,5):
#   df = pd.DataFrame(columns=['SourceText', 'Prediction', 'GroundTruth'])
#   n_correct = 0
#   n_total = 0
#   for seq_index in range(len(test_encoder_input_data)):
#       decoded_sentence = beam_search(test_encoder_input_data[seq_index:seq_index+1], beam_size=beamSize,target_token_index=target_token_index,
#                                      encoder_model=encoder_model,number_of_decoder_layers=3,reverse_target_char_index=reverse_target_char_index,
#                                      cell_type='gru',deocder_model=decoder_model,max_decoder_seq_length=max_decoder_seq_length )

#       if y_test[seq_index].strip() == decoded_sentence[0][0].strip():
#           n_correct += 1

#       n_total += 1
#       row = {}
#       row['SourceText'] = x_test[seq_index].strip()
#       row['GroundTruth'] = y_test[seq_index].strip()
#       row['Prediction'] = decoded_sentence[0][0].strip()
#       df = df.append(row, ignore_index=True)
#   df.to_csv('predictions_'+str(beamSize)+'.csv', index=False)  
#   test_accuracy[beamSize] = (n_correct * 100.0 / n_total)
# print('Test accuracy ', test_accuracy)

for beamSize in range(1):
  files.download('predictions_'+str(beamSize)+'.csv')
  time.sleep(30)